<a href="https://colab.research.google.com/github/AlmTechSoftware/piper/blob/research/training_data/research/feynman_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

## Dataset download from temporary 0x0.st

In [3]:
!curl http://0x0.st/HpsS.zip > dataset.zip
!ls dataset && rm -r dataset
!unzip dataset.zip -d dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.3M  100 11.3M    0     0  6556k      0  0:00:01  0:00:01 --:--:-- 6555k
ls: cannot access 'dataset': No such file or directory
Archive:  dataset.zip
   creating: dataset/valid/
  inflating: dataset/valid/20180910_153936_5b97bea9ddf2b358f90f884b_jpg.rf.bd21c8dd22ab72203e447876f98b8031.jpg  
  inflating: dataset/valid/EKF-UKF_jpg.rf.8806f6a948a6c61b96ed9d830d813753.jpg  
  inflating: dataset/valid/FqyewkjWwAE9ZsK_jpg.rf.dd1a8ab6b2d03fc455b512e8a6be8caf.jpg  
  inflating: dataset/valid/_annotations.coco.json  
  inflating: dataset/valid/D1D9CE52-103C-4BA1-A550-4F6F1018DC66_1_105_c-1_jpeg.rf.3ec235a322a99136a587c56039daa1db.jpg  
   creating: dataset/test/
  inflating: dataset/test/Adam-still_jpeg.rf.90fbaea30ed3e09fd7fda5aa92ea6a68.jpg  
  inflating: dataset/test/MPC_2_jpg.rf.8263fb9cbb0b86857de4519a380ac872.jpg  
  inflating:

## Dependencies

In [2]:
!pip install  opencv-python \
              scikit-image \
              python-dotenv \
              pycocotools \
              dataclasses-json \
              supervision \
              colored \
              wandb \
              torchvision \
              torchviz -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00


## WandB login

In [4]:
import wandb
wandb.login()

KeyboardInterrupt: ignored

# Definitions

## Dataset Handler Definition

In [5]:
LEARNING_RATE = 0.001
EPOCHS = 100
BATCH_SIZE = 8

In [6]:
!pip install pycocotools

In [21]:
import torchvision.transforms as T
_tensor_pil_transform = T.ToPILImage()

def tensor_to_pil(tensor):
    return _tensor_pil_transform(tensor)

In [29]:
import torch
from torch.utils.data import Dataset
from pycocotools.coco import COCO
from typing import Tuple, List
from PIL import Image
from collections import defaultdict as DD

import torchvision.transforms as transforms
import torchvision.transforms.functional as fun
import numpy as np

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import os

class CocoSegmentationDataset(Dataset):
    def __init__(self,
                 root_dir: str,
                 annotation_file: str,
                 do_transform: bool = True,
                 common_size: Tuple[int, int] = (512, 512),
                ):
        self.root_dir = root_dir
        self.common_size = common_size
        self.coco = COCO(annotation_file)
        self.image_ids = list(self.coco.imgs.keys())

        self.category_ids = self.coco.getCatIds()
        print(self.category_ids)
        self.num_categories = len(self.category_ids)

        if do_transform:
            self.transform = transforms.Compose([
                transforms.Resize(self.common_size),
                transforms.ToTensor(),
                # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

        self.normalize = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    def __len__(self):
        return len(self.coco.getImgIds())

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        image_info = self.coco.loadImgs(image_id)[0]
        image_path = os.path.join(self.root_dir, image_info['file_name'])
        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        annotations = self.coco.loadAnns(ann_ids)

        image = Image.open(image_path).convert("RGB")

        masks = np.zeros((len(annotations), image.height, image.width), dtype=np.uint8)
        for idx, ann in enumerate(annotations):
            mask = self.coco.annToMask(ann)
            masks[idx, :, :] = mask

        if self.transform:
            image = self.transform(image)
            # TODO: transform shitty masks??!??!?!?!?

        return image, masks


class Collater(object):
    def __init__(self):
        pass

    def __call__(self, batch):
        images, masks = zip(*batch)

        # Pad images and masks to match the dimensions of the largest image
        images_padded = pad_sequence(images, batch_first=True)
        masks_padded = pad_sequence(masks, batch_first=True)

        # Convert NumPy arrays to PyTorch tensors
        images_padded = torch.from_numpy(images_padded)
        masks_padded = torch.from_numpy(masks_padded)

        return images_padded, masks_padded

dataset = CocoSegmentationDataset(root_dir="dataset/train/",
                                  annotation_file="dataset/train/_annotations.coco.json")

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=Collater())

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
[0, 1, 2]


In [30]:
for batch in dataloader:
    imgs, masks = batch
    print(f"{imgs.shape=} {masks.shape=}")
    display(tensor_to_pil(imgs[0]))
    break

TypeError: ignored

#### Custom dataset handler

In [ ]:
import os
import torch
import numpy as np

from torch import Tensor
from torch.utils.data import Dataset
from torchvision import transforms

from PIL import Image

from typing import Tuple

from glob import glob

class PNGMaskDataset(Dataset):
    def __init__(
        self,
        dataset_dir: str,
        shape_dim: Tuple[int, int] = (512, 512),
        do_shape_dim: bool = True,
    ):
        self.dataset_dir = dataset_dir
        self.shape_dim = shape_dim

        if do_shape_dim:
            self.transform = transforms.Compose(
                [
                    transforms.Resize(self.shape_dim), # sadly needed due to stacking
                    transforms.ToTensor(),
                ]
            )
        else:
            self.transform = transforms.Compose([transforms.ToTensor()])

        # Example _classes.csv
        """
        Pixel Value, Class
        0, background
        1, sghug
        2, w%
        """

        # Parse _classes.csv
        fh = open(os.path.join(self.dataset_dir, "_classes.csv"))
        lines = fh.readlines()
        fh.close()

        self.classes = map(lambda csv: csv.split(", "), lines)
        self.classes = map(lambda csv: (csv[1], int(csv[0])), self.classes)


        # Load images & masks
        self.mask_files = glob(os.path.join(self.dataset_dir, "*_mask.png"))

        all_files =  glob(os.path.join(self.dataset_dir, "*"))
        self.images = {}
        for mask_file in self.mask_files:
            img_file = mask_file.replace("_mask.png", ".jpg")
            base, ext = os.path.splitext(img_file)

            if img_file in all_files:
                self.images[base] = {
                    "image": img_file,
                    "mask": mask_file,
                }

    def __str__(self):
        return str(self.images)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_info = list(self.images.values())[idx]
        image_path, mask_path = image_info["image"], image_info["mask"]

        image = Image.open(image_path)
        mask = Image.open(mask_path)

        image = self.transform(image)
        mask = self.transform(mask)

        return image, mask

##### Example

In [ ]:
# dataset = PNGMaskDataset("dataset/train")

img, mask = dataset[0]
img2, mask2 = dataset[2]

display(torch.max(mask))
display(torch.max(mask2))

img = tensor_to_pil(img)
mask = tensor_to_pil(mask * 40)

img2 = tensor_to_pil(img2)
mask2 = tensor_to_pil(mask2 * 40)

display(img)
display(mask)
display(img2)
display(mask2)

del dataset, img, mask, img2, mask2

ValueError: ignored

#### Roboflow dataset handler

In [ ]:
!pip install roboflow

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install roboflow

In [ ]:
from roboflow import Roboflow
import os

key_path = "/content/drive/MyDrive/roboflow_api_key.txt"
try:
    fh = open(key_path, "r")
    api_key = fh.readline().rstrip()
except KeyError as err:
    print(f"Unable to find Roboflow API key in '{key_path}'")
    raise err

rf = Roboflow(api_key=api_key)
project = rf.workspace("almqv").project("feynman")
dataset = project.version(7).download("coco-segmentation")
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to FeynMAN-7 in coco-segmentation:: 100%|██████████| 88/88 [00:00<00:00, 751.84it/s]


TypeError: ignored

## FeynMAN Model Definition

In [7]:
import torch
import torch.nn as nn

from torch import Tensor

class FeynMAN(nn.Module):
    def __init__(self, in_channels: int = 3, num_classes: int = 2):
        super(__class__, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

        self.classifier = nn.Conv2d(256, num_classes, kernel_size=1)

    def forward(self, x: torch.Tensor) -> Tensor:
        x = self.conv_layers(x)
        x = self.classifier(x)
        return x

# Training

In [8]:
!nvidia-smi && echo "YES YOU CAN TRAIN WITH CUDA!"

Wed Aug 30 19:12:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim

def train_model(
    model: nn.Module,
    dataloader: DataLoader,
    epochs: int = EPOCHS,
    batch_size: int = BATCH_SIZE,
    learning_rate: float = LEARNING_RATE,
    device: torch.device = torch.device("cpu"),
):
    # start a new wandb run to track this script
    wandb.init(
        # set the wandb project where this run will be logged
        project="feynman_2",

        # track hyperparameters and run metadata
        config={
        "learning_rate": learning_rate,
        "epochs": epochs,
        "batch_size": batch_size,
        "architecture": "FeynMAN",
        "dataset": "feynman_v7i",
        }
    )

    # Watch the model for cool weights & biases
    wandb.watch(model)

    # criterion = nn.CrossEntropyLoss() # nn.BCELoss() #### TODO: change?
    criterion = nn.BCELoss()
    params = model.parameters()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # dataset = PNGMaskDataset(dataset_dir)
    # dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    print("BEGIN TRAINING!")
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        print(epoch)

        for batch_idx, (images, masks) in enumerate(dataloader):
            images, masks = images.to(device), masks.to(device) #.float()
            print("#", batch_idx)

            # Forward pass
            outputs = model(images)
            print("post forward pass")
            loss = criterion(outputs, masks)
            print("loss done")

            # Backpropagation and optimization
            optimizer.zero_grad()
            print("zero grad done")
            loss.backward()
            print("backward done")
            optimizer.step()
            print("step done")

            running_loss += loss.item()
            wandb.log({"Loss": loss})

            print(
                f"Epoch [{epoch+1}/{epochs}] Batch [{batch_idx+1}/{len(dataloader)}] - Loss: {loss:.4f}"
            )

        wandb.log({"Running loss": running_loss})

    print("END TRAINING!")
    wandb.finish()

### Model creation

In [10]:
model = FeynMAN()
device_type = "cuda" if torch.cuda.is_available() else "cpu"
#device_type = "cpu"
device = torch.device(device_type)

# Put the model on the GPU
model = model.to(device)

#### Debug

In [ ]:
# Test the input etc.

input_height = 256
input_width = 256
random_input = torch.randn(1, 3, input_height, input_width)

# Forward pass
segmentation_map = model(random_input)
print("Segmentation map shape:", segmentation_map.shape)

Segmentation map shape: torch.Size([1, 3, 256, 256])


In [ ]:
from torchviz import make_dot

ds = PNGMaskDataset("./dataset/test/")
x_tmp, _ = ds[0]
x_tmp = x_tmp.to(device)
y_tmp = model(x_tmp.unsqueeze(0))

make_dot(y_tmp, params=dict(list(model.named_parameters()))).render("model.png", format="png")

del model, ds, x_tmp, y_tmp, _

In [ ]:
!pip install torchsummary


model = model.to(device)

from torchsummary import summary
summary(model, input_size=(3, 128, 128), batch_size=8, device="cuda")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [8, 64, 128, 128]           1,792
              ReLU-2          [8, 64, 128, 128]               0
            Conv2d-3         [8, 128, 128, 128]          73,856
              ReLU-4         [8, 128, 128, 128]               0
            Conv2d-5         [8, 256, 128, 128]         295,168
              ReLU-6         [8, 256, 128, 128]               0
            Conv2d-7           [8, 3, 128, 128]             771
Total params: 371,587
Trainable params: 371,587
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.50
Forward/backward pass size (MB): 899.00
Params size (MB): 1.42
Estimated Total Size (MB): 901.92
----------------------------------------------------------------


### Start training loop

#### FCN ResNet 50 test

In [ ]:
from torchvision.models.segmentation import fcn_resnet50

model = fcn_resnet50(num_classes=3, pretrained=False)
model = model.to(device)

#### Train the model

In [11]:
# Fix for some CUDA bullshit

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [20]:
print(f"Doing training on device type '{device_type}'!")
torch.cuda.empty_cache()
train_model(model, dataloader=dataloader, device=device, batch_size=BATCH_SIZE, learning_rate=LEARNING_RATE, epochs=EPOCHS)

Doing training on device type 'cuda'!


BEGIN TRAINING!
0


KeyError: ignored